# Fome Zero Foods - Visão Restaurantes

Neste notebook vamos explorar o dataset de restaurantes da Fome Zero Foods, respondendo a perguntas de negócios sobre o contexto da visão restaurantes do dataset.

## 1. Importando bibliotecas e criando Dataframe

### 1.1 Importando bibliotecas

In [1]:
import pandas as pd
import plotly.express as px

from pathlib import Path

### 1.2 Criando Dataframe

In [2]:
# caminho do arquivo
path = Path.cwd().parent / 'data' / 'processed' / 'dataset_processed.csv'

try:
    with open(path, 'r', encoding='utf-8') as arquivo:
        df = pd.read_csv(arquivo)
except FileNotFoundError:
    print(f"Arquivo não encontrado: {path}")

In [3]:
df.head(3)

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes,country_name,price_type,color_name
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,Philippines,expensive,darkgreen
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,Philippines,expensive,darkgreen
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,0,4,4.7,3F7E00,Excellent,469,Philippines,gourmet,darkgreen


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7512 entries, 0 to 7511
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   restaurant_id         7512 non-null   int64  
 1   restaurant_name       7512 non-null   object 
 2   country_code          7512 non-null   int64  
 3   city                  7512 non-null   object 
 4   address               7512 non-null   object 
 5   locality              7512 non-null   object 
 6   locality_verbose      7512 non-null   object 
 7   longitude             7512 non-null   float64
 8   latitude              7512 non-null   float64
 9   cuisines              7512 non-null   object 
 10  average_cost_for_two  7512 non-null   int64  
 11  currency              7512 non-null   object 
 12  has_table_booking     7512 non-null   int64  
 13  has_online_delivery   7512 non-null   int64  
 14  is_delivering_now     7512 non-null   int64  
 15  price_range          

## 2. Perguntas de Negócio

### 2.1 Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [5]:
restaurante = (
    df.loc[:, ['restaurant_name', 'aggregate_rating']]
    .groupby('restaurant_name')
    .count()
    .sort_values(by='aggregate_rating', ascending=False)
    .reset_index()
)
print(f"Restaurante com maior qtd avaliações: {restaurante.iloc[0,0]}")

Restaurante com maior qtd avaliações: Domino's Pizza


### 2.2 Qual o nome do restaurante com a maior nota média?

In [6]:
restaurante = (
    df.groupby('restaurant_name')[['aggregate_rating', 'restaurant_id']]
    .agg(agg_rating_mean=('aggregate_rating', 'mean'), rest_id_min=('restaurant_id', 'min'))
    .sort_values(by=['agg_rating_mean', 'rest_id_min'], ascending=[False, True])
    .reset_index()
)

print(f"Restaurante com maior nota média: {restaurante.iloc[0,0]}")

Restaurante com maior nota média: Indian Grill Room


### 2.3 Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [7]:
restaurante = (
    df.groupby('restaurant_name')[['average_cost_for_two', 'restaurant_id']]
    .agg(cost_for_two_max=('average_cost_for_two', 'max'), rest_id_min=('restaurant_id', 'min'))
    .sort_values(by=['cost_for_two_max', 'rest_id_min'], ascending=[False, True])
    .reset_index()
)

print(f"Restaurante com maior valor do prato para duas pessoas: {restaurante.iloc[0,0]}")

Restaurante com maior valor do prato para duas pessoas: d'Arry's Verandah Restaurant


### 2.4 Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [8]:
restaurante = (
    df.loc[df['cuisines']=='Brazilian', ['restaurant_name', 'aggregate_rating', 'restaurant_id']]
    .groupby('restaurant_name')
    .min()
    .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=True)
    .reset_index()
)

print(f"Restaurante de culinária brasileira com menor avaliação: {restaurante.loc[0, 'restaurant_name']}")

Restaurante de culinária brasileira com menor avaliação: Loca Como tu Madre


### 2.5 Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [9]:
filtro = (df['cuisines']=='Brazilian') & (df['country_name']=='Brazil')
restaurante = (
    df.loc[filtro, ['restaurant_name', 'aggregate_rating', 'restaurant_id']]
    .groupby('restaurant_name')
    .agg(mean_rating=('aggregate_rating', 'mean'), rest_id_min=('restaurant_id', 'min'))
    .sort_values(by=['mean_rating', 'rest_id_min'], ascending=[False, True])
    .reset_index()
)

print(f"Restaurante brasileiro, de culinária brasileira com maio media de avaliação: {restaurante.iloc[0, 0]}")

Restaurante brasileiro, de culinária brasileira com maio media de avaliação: Braseiro da Gávea


### 2.6 Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [10]:
# Verificação de linha sem nota 0 ou sem nota
filtro = df['aggregate_rating']==0
nota0 = df.loc[filtro, 'aggregate_rating'].count()

# Verificação de linha sem nota
filtro2 = df['rating_text']=='Not rated'
nota_text = df.loc[filtro2, 'rating_text'].count()

print("Verificação de colunas sem nota!")
print(f"Coluna 'aggregate_rating' possuí {nota0} linhas com nota 0!")
print(f"Coluna 'rating_text' possuí {nota_text} linhas com nota 'Not rated'!")
print("Indentifição confirmada de linhas com dados sem nota!")

Verificação de colunas sem nota!
Coluna 'aggregate_rating' possuí 121 linhas com nota 0!
Coluna 'rating_text' possuí 121 linhas com nota 'Not rated'!
Indentifição confirmada de linhas com dados sem nota!


In [11]:
# Condição de filtro para retirar linhas que não possuem nota atribuida
filtro = df['aggregate_rating']>2
# Dataframe para contagem de entregas
df_aux = df.loc[filtro, 'has_online_delivery'].value_counts().reset_index()

var1 = 0
var2 = 0
for index, row in df_aux.iterrows():
    if (index==0):
        var1 = row['count']
        print(f"Restautantes que não realizam entregas possuem {row['count']} entregas registradas!")
    elif (index==1):
        var2 = row['count']
        print(f"Restautantes que realizam entregas possuem {row['count']} entregas registradas!")

if (var1 > var2):
    print(f"\nResposta: Restaurantes que não aceitam entregas possuem mais avaliações registradas!")
elif (var1 < var2):
    print(f"\nResposta: Restaurantes que aceitam entregas possuem mais avaliações registradas!")
else:
    print(f"\nResposta: Restaurantes que aceitam e não aceitam entregas, possuem a mesma quantidade de avaliações registradas!")

Restautantes que não realizam entregas possuem 4755 entregas registradas!
Restautantes que realizam entregas possuem 2636 entregas registradas!

Resposta: Restaurantes que não aceitam entregas possuem mais avaliações registradas!


### 2.7 Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [12]:
# Média dos restaurantes que NÃO fazem resversas
rest_no_table_booking = df.loc[df['has_table_booking']==0, 'average_cost_for_two'].mean()
# Média dos restaurantes que fazem reservas
rest_table_booking = df.loc[df['has_table_booking']==1, 'average_cost_for_two'].mean()

print(f"Restaurantes que não fazem reservas: {rest_no_table_booking:,.2f} de valor médio para um prato para duas pessoas!")
print(f"Restaurantes que fazem reservas: {rest_table_booking:,.2f} de valor médio para um prato para duas pessoas!")

Restaurantes que não fazem reservas: 3,460.61 de valor médio para um prato para duas pessoas!
Restaurantes que fazem reservas: 62,931.51 de valor médio para um prato para duas pessoas!


In [13]:
if (rest_no_table_booking > rest_table_booking):
    print(f"Restaurantes que não fazem reservas são, os restaurantes que possuem o maior valor médio de um prato para duas pessoas!")
elif (rest_no_table_booking < rest_table_booking):
    print(f"Restaurantes que fazem reservas são, os restaurantes que possuem o maior valor médio de um prato para duas pessoas!")
else:
    print(f"Restaurantes que fazem reservas ou não, possuem o mesmo valor médio de um prato para duas pessoas!")

Restaurantes que fazem reservas são, os restaurantes que possuem o maior valor médio de um prato para duas pessoas!


### 2.8 Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [14]:
filtro = (df['country_name']=='United States of America') & (df['cuisines'].isin(['BBQ', 'Japanese']))
df_aux = df.loc[filtro, ['cuisines', 'average_cost_for_two']]
df_rest = df_aux.groupby('cuisines').mean().reset_index()

df_rest

,cuisines,average_cost_for_two
0,BBQ,39.642857
1,Japanese,56.406250


In [15]:
# --- Lógica para texto dinâmico ---
# Extraindo os valores para variáveis
custo_bbq = df_rest.loc[df_rest['cuisines'] == 'BBQ', 'average_cost_for_two'].values[0]
custo_jap = df_rest.loc[df_rest['cuisines'] == 'Japanese', 'average_cost_for_two'].values[0]

# Comparação e montagem da string
if custo_jap > custo_bbq:
    print(f"A culinária Japonesa tem o maior custo médio ${custo_jap:.2f}, sendo superior as Churrascarias Americanas (BBQ) de ${custo_bbq:.2f}!")
elif custo_jap < custo_bbq:
    print(f"A culinária Churrascarias Americanas (BBQ) tem o maior custo médio ${custo_bbq:.2f}, sendo superior à Japonesa de ${custo_jap:.2f}!")
else:
    print(f"As culinárias Churrascarias Americanas (BBQ) e Japonesa possuem o mesmo custo médio! (${custo_bbq:.2f})")

A culinária Japonesa tem o maior custo médio $56.41, sendo superior as Churrascarias Americanas (BBQ) de $39.64!


In [ ]:
# --- Construção do Gráfico Plotly ---
# Criando uma coluna de cor para destacar o maior valor automaticamente
df_rest['Legenda:'] = df_rest['average_cost_for_two'].apply(
    lambda x: 'Maior Custo Medio' if x == df_rest['average_cost_for_two'].max() else 'Menor Custo Medio'
)

# Criando o gráfico de barras
fig = px.bar(
    df_rest,
    x='cuisines',
    y='average_cost_for_two',
    color='Legenda:',  # Usa a coluna criada para definir as cores
    color_discrete_map={'Maior Custo Medio': 'red', 'Menor Custo Medio': 'gray'}, # Define as cores especificas
    text_auto='.2f',   # type: ignore # Mostra o valor em cima da barra
    title='Custo Médio: BBQ vs Japonesa (EUA)',
    labels={'cuisines': 'Culinária', 'average_cost_for_two': 'Preço Médio para Dois ($)'}
)

# Ajustes finais de layout (opcional)
#fig.update_layout(showlegend=False) # Remove a legenda se desejar limpar o visual
fig.show()